In [3]:
with open('/home/student/dbanalysis/dbanalysis/resources/dep_times.pickle','rb') as handle:
    import pickle
    d = pickle.load(handle)

In [4]:
import json
routes = json.loads(open('/home/student/dbanalysis/dbanalysis/resources/trimmed_routes.json').read())

In [9]:
len(routes['25'])

2

In [21]:
routes['39A'][4]

[' Ongar',
 328,
 1443,
 1444,
 1445,
 1647,
 1648,
 1649,
 1911,
 1913,
 1914,
 1805,
 1806,
 1660,
 1661,
 1662,
 1664,
 1665,
 1666,
 1807,
 7167,
 1808,
 7389,
 7025,
 4464,
 1869,
 1870,
 1871,
 1872,
 1873,
 1874,
 1875,
 1876,
 1877,
 1878,
 1879,
 1899,
 6107,
 6108,
 6109,
 6110,
 7020,
 7029,
 7038,
 7011,
 2171,
 7160,
 7047,
 7161,
 7162]

In [27]:
a= set(routes['39A'][0])
b=routes['39A'][4]

In [28]:
for i in a:
    if i not in b:
        print(i)

768
769
770
771
772
773
774
775
776
777
779
780
781
782
783
784
785
786
793
7586
7587
7588
767


In [8]:
from dbanalysis import stop_tools
import pandas as pd
import numpy as np
class BRModel():
    """
    Big route model class
    uses the distance of a stop from first stop on a route to compute predictions lalalala
    MAPE and r2 scores are not as good as they were in the notebook
    (we achieved 0.57 r2, and 7% MAPE on the time to complete the route)
    Should look into this.
    Planned time of arrival boosts score considerably
    """
    def __init__ (self, route,variation,verbose=True,src='build',rgr='RandomForest',\
                mode='validate',features = ['day','base_time_dep','distance','vappr'],use_dummies=True):
        
        import json
        self.regr_type = rgr
        self.verbose = verbose
        self.route = route
        self.use_dummies = use_dummies
        self.variation = variation
        self.routes = json.loads(open('/home/student/dbanalysis/dbanalysis/resources/trimmed_routes.json').read())
        self.features = features
        
        self.route_array = self.routes[route][variation][1:]
        
        if src == 'build':
            if not self.can_be_modelled():
                print('fuck')
                raise ValueError ('Missing data for modelling this route')

            self.gather_data()
            self.preprocess()
            if rgr == 'RandomForest':
                from sklearn.ensemble import RandomForestRegressor as rf
                self.rgr = rf()
            elif rgr == 'Linear':
                from sklearn.linear_model import LinearRegression as lr
                self.rgr = lr(fit_intercept=True)
            elif rgr == 'Neural':
                from sklearn.neural_network import MLPRegressor as mlpr
                self.rgr = mlpr(hidden_layer_sizes=(80,80,80,80,80,80),
                                activation='tanh')
                del(self.routes)
                if mode == 'validate':
                    self.validate_neural_network()
                elif mode == 'build':
                    self.build_neural_network()
                elif mode == 'just data':
                    pass
    
    
    def build_neural_network(self):
        import numpy as np
        msk = np.random.rand(len(self.data)) < 0.7
        self.train = self.data[msk]
      
        self.test = self.data[~msk]
        del(msk)
        
        from sklearn.preprocessing import MinMaxScaler as mms
        del(self.data)
        self.X_transformer = mms().fit(self.train[self.features])
        Y = self.train['traveltime'].values
        Y=Y.reshape(-1,1)
        self.Y_transformer = mms().fit(Y)
        X = self.X_transformer.transform(self.train[self.features])
        Y = self.Y_transformer.transform(Y)
        
        self.model = self.rgr.fit(X,Y)
        print('Built')
        del(X)
        del(Y)
        del(self.train)
   
               
    def build_model():
        import numpy as np
        if self.verbose:
            print('building model')
        #how many rows are we going to train on. Here we just randomly pick 50% of the samples and use that. Full df is probably too large
        msk = np.random.rand(len(self.data)) < 0.5
        self.data = self.data[msk]
        del(msk)
        self.model = self.rgr.predict(self.data[features].values,self.data[msk].values)
        del(self.data)
    def dump_model(self):
        import pickle
        import time
        t = int(time.time())
        name = str(self.route) + '_' + str(self.variation) + '_' + str(time.time()) +'.bin'
        with open('/data/BRModels/models/'+name,'wb') as handle:
            pickle.dump(self,handle,protocol = pickle.HIGHEST_PROTOCOL) 
        print('Saved Model')
    def predict(self,X):
        return self.model.predict(X)



    def gather_data(self):
        if self.verbose:
            print('gathering data...')
        from dbanalysis import stop_tools
        arr = self.route_array
        import os
        to_concat = []
        for i in range(len(arr)-1):
            
            data = stop_tools.get_stop_link(arr[i],arr[i+1])
            if data is not None:
                
            
                routeids = data['routeid'].unique()
                valid_routeids = [r for r in routeids if r.split('_')[0] == self.route]
                data = data[data['routeid'].isin(valid_routeids)]
                #data = data[data['actualtime_arr_to']>data['actualtime_arr_from']]
                data['traveltime']=data['actualtime_arr_to'] - data['actualtime_arr_from']
                print(arr[i],arr[i+1])
                print(data['traveltime'].min(),data['traveltime'].mean(),\
                      data['traveltime'].quantile(q=0.05))
                data = data[data['traveltime']>data['traveltime'].quantile(q=0.05)]
                to_concat.append(data)
            
            del(data)
        self.data = pd.concat(to_concat,axis=0)
        del to_concat
        

    def preprocess(self):
        if self.verbose:
            print('Preprocessing data')
        self.select_routes()
        self.clean_1()
        self.add_distances()
        
        self.add_base_departure_time()
        self.add_time_info()
        self.data = self.data[self.data['traveltime']>0]
        #self.data = self.data[~self.data[['dayofservice','tripid']].isin(rubbish[['dayofservice','tripid']])]
        self.merge_weather()
        
        #self.add_dummies()
        #self.features += self.dummy_features
        self.data = self.data[self.data['traveltime']>0]
        #self.data['planned_traveltime'] = self.data['plannedtime_arr_from'] - self.data['base_time_dep']
    def select_routes(self):
        if self.verbose:
            print('parsing routeids')
        routeids = self.data['routeid'].unique()
        valid_routeids = [r for r in routeids if r.split('_')[0] == self.route]
        self.data = self.data[self.data['routeid'].isin(valid_routeids)]
    def clean_1(self):
        if self.verbose:
            print('dropping null values')
        self.data = self.data.dropna()
        
    def add_distances(self):
        if self.verbose:
            print('adding distances')
        s_getter =stop_tools.stop_getter()
        total_distance = 0
        r = self.route_array
        route_distances = {r[0]:0}
        
        for i in range(0, len(r)-1):
            distance = s_getter.get_stop_distance(str(r[i]),str(r[i+1]))
                
            total_distance += distance
            route_distances[r[i+1]]=total_distance
        self.data['distance']=self.data['stopA'].apply(lambda x: route_distances[x])
        del(s_getter)
    def add_base_departure_time(self):
        if self.verbose:
            print('adding base departure times')
       
        keys= self.data[self.data['stopA']==self.route_array[0]]
        keys['base_time_dep']=keys['actualtime_dep_from']
        keys2=keys[['tripid','dayofservice','base_time_dep']]
        self.data = pd.merge(self.data,keys2,on=['dayofservice','tripid'])
        
        self.data['traveltime']=self.data['actualtime_dep_from']-self.data['base_time_dep']        
        del(keys)
        del(keys2)
    
    def add_time_info(self):
        if self.verbose:
            print('adding time information')
        time_format = "%d-%b-%y %H:%M:%S"
        self.data['dt']=pd.to_datetime(self.data['dayofservice'],format=time_format)
        self.data['day']=self.data['dt'].dt.dayofweek
        self.data['month']=self.data['dt'].dt.month
        self.data['hour'] = self.data['base_time_dep'] //3600
        self.data['weekend']=self.data['day']>4
        self.data['year']=self.data['dt'].dt.year
        self.data['date'] = self.data['dt'].dt.date
    def merge_weather(self,weather=None):
        if self.verbose:
            print('merging weather')
        if weather == None:
          
            weather = pd.read_csv('/home/student/dbanalysis/dbanalysis/resources/cleanweather.csv').dropna()
        weather['dt']=pd.to_datetime(weather['date'])
        weather['hour']=weather['dt'].dt.hour
        weather['date']=weather['dt'].dt.date
        
        self.data = pd.merge(self.data,weather,on=['date','hour'])
        del(weather)

    def add_dummies(self):
        
        if self.verbose:
            print('Making dummy features')
        self.data = pd.get_dummies(self.data,columns=['day','hour','month'])
        for hour in range(5,25):
            if 'hour_'+str(hour) not in self.data.columns:
                self.data['hour_'+str(hour)] = 0

        self.dummy_features = [col for col in self.data.columns\
                                if (col[0:3] == 'day' and col != 'dayofservice')\
                                or col[0:5] == 'month' or col[0:4] == 'hour']
       
    def can_be_modelled(self):
        if self.verbose:
            print('Checking for data files')
        import os
        base_dir = '/data/stops/'
        arr = self.route_array
        #if the first stop is missing, then we won't attempt to model this route
            
        if os.path.exists(base_dir+str(arr[1])+'/'+str(arr[2])+'.csv'):

            return True
        else:
            return False

In [9]:
import time
t1=time.time()
r=BRModel('15',1,mode='build',rgr='Neural')
print(time.time()-t1)

Checking for data files
gathering data...
6318 6319
0 23.830361026533275 16.0
6319 7246
1 41.01663246570904 22.0
7246 6320
28 89.62953478446437 55.0
6320 4594
14 86.69421094369548 45.0
4594 4595
44 91.5393702863773 57.0
4595 4596
8 47.848928439787265 17.0
4596 4563
21 63.78592452433512 32.0
4563 1218
17 79.28586464582675 40.0
1218 1270
28 64.99826498422713 38.0
1270 1272
28 65.02957924707371 34.0
1272 1273
14 45.896142902307936 21.0
1273 1274
16 84.148198718622 29.0
1274 1275
27 66.00204423943809 35.0
1275 1276
4 24.53630406290957 9.0
1276 1277
22 51.86056884970613 30.0
1277 1219
10 43.81562106588334 16.0
1219 1220
22 59.523002041136756 31.0
1220 1221
18 58.606127373832756 26.0
1221 664
19 70.40759559979047 28.0
664 665
14 50.13939140437101 22.0
665 666
14 36.81618684361774 20.0
666 667
15 38.67224517546153 22.0
667 668
26 71.62059946644348 35.0
668 614
17 80.59057668198913 25.0
614 615
19 50.017826426896015 27.0
615 616
14 56.806362306727955 24.0
616 617
11 57.01498265166649 24.0
617 

/home/student/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:175: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


adding time information
merging weather


/home/student/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/home/student/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Built
186.15736484527588


In [10]:
X = r.X_transformer.transform(r.test[r.features])
import numpy as np
msk = np.random.rand(len(X)) <0.2
Y=r.model.predict(X)
a = r.Y_transformer.inverse_transform(Y.reshape(-1,1))
count1=0
count2=0
for i in a:
    if i > 0:
        count1+=1
    else:
        count2+=1
        

In [11]:
print(count1,count2)

304881 8900


In [12]:
r.test['real']=a

In [13]:
from sklearn import metrics
metrics.mean_absolute_error(r.test['traveltime'],r.test['real'])

216.80543464959158

In [14]:
((abs(r.test['traveltime']-r.test['real'])/r.test['traveltime'])*100).mean()

18.23447196249181

In [207]:
thing=r.test[r.test['real']<0]

In [208]:
thing['distance'].unique()

array([0.19439372, 0.42661594, 0.91399099])

In [156]:
thing['month'].unique()

KeyError: 'month'

In [157]:
thing['day'].unique()

KeyError: 'day'

In [126]:
thing['hour'].unique()

array([22, 21, 18,  7,  8, 17, 14, 11, 19, 20, 16, 12, 13,  9, 15, 10,  6,
       23,  5])

In [127]:
thing2=r.test[r.test['real']>0]
thing2['distance'].unique()

array([ 2.13710899,  2.73836243,  4.60077595,  8.459664  ,  9.06522538,
        9.63199619, 12.14116219, 13.38822997, 14.33519184, 14.54172718,
       15.38787347, 16.06248638, 18.76220611, 19.600695  , 21.35775595,
       22.60855934,  3.47239308,  5.52456992,  6.80926407,  9.34944473,
        9.91663643, 13.9437401 , 14.10590383, 17.86033059, 20.46557701,
       22.28254239,  3.98918733,  5.73902979, 13.00315193, 19.16280228,
       19.86110042, 20.63406303, 20.9327889 ,  2.41355696,  3.05509448,
        7.12874632,  7.73113298, 11.79891293, 15.17371397, 15.87786843,
        2.12746783,  5.07080026,  7.41718128, 12.56124053, 15.67475222,
       16.90819453, 17.16837317, 17.39439661, 18.30079427,  6.47090175,
       10.38606964, 22.17246364,  4.29713731,  5.22224552,  8.79233752,
       13.59259852, 16.61976575, 14.83736879, 10.07682693,  8.15250246,
       16.37022232,  6.13097876, 13.8187145 , 11.02329844, 11.24524183])

In [93]:
r.test[r.test['pred']<0]['distance'].unique()

array([0.42661594, 0.19439372, 0.91399099, 2.12746783, 2.13710899,
       2.41355696, 2.73836243, 3.05509448])

In [99]:
r.test[r.test['pred']<0].columns

Index(['index', 'dayofservice', 'tripid', 'plannedtime_arr_from',
       'plannedtime_dep_from', 'actualtime_arr_from', 'actualtime_dep_from',
       'plannedtime_arr_to', 'actualtime_arr_to', 'routeid', 'stopA', 'stopB',
       'traveltime', 'distance', 'base_time_dep', 'dt_x', 'weekend', 'year',
       'date', 'dewpt', 'msl', 'rain', 'rhum', 'temp', 'vappr', 'wetb', 'dt_y',
       'day_0', 'day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6', 'hour_5',
       'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'month_1',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'hour_24',
       'planned_traveltime', 'pred'],
      dtype='object')

In [90]:
count2

12927

In [67]:
from dbanalysis.classes import time_tabler_refac as time_tabler
import datetime
dt = datetime.datetime.now()
t=time_tabler.time_tabler()
d=t.get_dep_times_five_days('15',dt)
df = d[1]['matrix']
import pandas as pd
df = pd.get_dummies(df,columns=['day','hour'])
df['vappr']=10.0
for f in r.features:
    if f not in df.columns:
        df[f]=0
X = r.X_transformer.transform(df[r.features])
Y=r.model.predict(X)
a = r.Y_transformer.inverse_transform(Y.reshape(-1,1))
count1=0
count2=0
for i in a:
    if i > 0:
        count1+=1
    else:
        count2+=1

In [68]:
count1

30242

In [69]:
count2

1636

In [16]:
print(min(a))

[-866.23331319]
